In [1]:
from EMAN3 import *
from EMAN3tensor import *
import numpy as np
tf_set_device(dev=0,maxmem=4096)

2024-04-28 09:31:14.232886: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 09:31:15.849400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 09:31:15.864022: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#

In [2]:
orts=Orientations(np.array([[0,0,0],[0,0,.25],[0,0,.5],[0,0,.75],[0.125,0,0],[0.25,0,0],[0.375,0,0],[0.5,0,0]]))

In [3]:
orts.to_mx2d()

<tf.Tensor: shape=(2, 3, 8), dtype=float32, numpy=
array([[[ 1.0000000e+00, -1.1920929e-07, -1.0000000e+00, -1.1920929e-07,
          1.0000000e+00,  1.0000000e+00,  1.0000000e+00,  1.0000000e+00],
        [ 0.0000000e+00, -1.0000000e+00,  8.7422777e-08,  1.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00]],

       [[ 0.0000000e+00,  1.0000000e+00, -8.7422777e-08, -1.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 1.0000000e+00, -1.1920929e-07, -1.0000000e+00, -1.1920929e-07,
          7.0710671e-01, -1.1920929e-07, -7.0710695e-01, -1.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         -7.0710683e-01, -1.0000000e+00, -7.0710683e-01,  8.7422777e-08]]],
      dtype=float32)>

In [3]:
gaus=Gaussians(np.array([[0.0,0.0,0.0,1.0],[.3,0,0,1],[0,.3,0,.5],[0,0,.3,.25],[0,0,-.1,.25]]))

In [5]:
imgs=gaus.project_simple(orts,64)

2024-02-03 20:23:07.132239: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [6]:
emimgs=from_tf(imgs,True)

In [7]:
imgsf=tf_fft2d(imgs)

In [10]:
tf_frc(imgsf[:2],imgsf[2:4],True)

(46,) (64, 33, 1) (2, 64, 33)
(46,) (64, 33, 1) (2, 64, 33)


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.7433562, 0.7350201], dtype=float32)>

In [ ]:
ptcls=EMData.read_images("/home2/stevel/test3d/ptcls_08_even.hdf")
xfs=[ptcl["xform_projection"] for ptcl in ptcls]

In [3]:
type(np.zeros(5,dtype="float32"))

numpy.ndarray

In [2]:
N=8
x=EMStack2D([test_image(size=(1024,1024)) for i in range(N)])
for i in range(N): x.emdata[i].translate(i*3,i*2,0)

In [4]:
xf=x.do_fft()
cf=xf.calc_ccf(xf[0],center=True)
#cfp=tf_phaseorigin2d(cf.tensor)
cfr=cf.do_ift()
EMData.write_images("ccf.hdf",cfr.emdata)

True

In [4]:
tf.complex64

tf.complex64

In [5]:
x[0].div

<tf.Tensor: shape=(1024, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [6]:
# Developing init_from_map method
emd=test_image_3d()
seg=emd.process("segment.gauss",{"minratio":0.1,"width":4,"skipseg":1})
amps=np.array(seg["segment_amps"])
centers=np.array(seg["segment_centers"]).reshape(len(amps),3)
centers/=(emd["nx"],emd["ny"],emd["nz"])
centers-=(0.5,0.5,0.5)
amps/=max(amps)
print(centers.shape,amps.shape)
cc=np.concatenate((centers.transpose(),amps.reshape((1,len(amps)))))
print(cc.shape,cc[0])
#

(168, 3) (168,)
(4, 168) [ 0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.4375
 -0.4375    -0.375      0.375     -0.34375    0.3359375 -0.3125
 -0.4921875  0.4921875  0.296875  -0.28125    0.2578125 -0.25
 -0.2109375  0.2109375 -0.1796875  0.171875  -0.1484375  0.1328125
 -0.1171875  0.09375   -0.0859375 -0.0546875  0.0546875  0.
  0.         0.         0.         0.         0.        -0.40625
  0.40625    0.         0.         0.         0.         0.
  0.        -0.4609375  0.46875    0.         0.         0.03125
  0.         0.         0.234375  -0.0390625  0.

In [2]:
# Testing orientation conversion and making sure projection directions match
# This is also necessary to initialize the test code below for training Gaussian locations based on FRC

map1=EMData("A.hdf")
gaus=Gaussians()
gaus.init_from_map(map1,4,.5)
print(len(gaus)," gaussians init")

ptcls=EMStack2D("A_proj.hdf")
orts,txty=ptcls.orientations

projs=gaus.project_simple(orts,map1["nx"],txty=txty)
#projs.write_images("A_reproj.hdf")
projsf=projs.do_fft()
ptclsf=ptcls.do_fft()


524  gaussians init


2024-04-28 09:31:22.082688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [18]:
frcs=tf.stack(tf_frc(projsf.tensor,ptclsf.tensor,0))
tf.math.reduce_mean(frcs[:,:10],1)

In [16]:
gaus[:10],gaus[-10:]

(<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
 array([[ 0.12574698, -0.01763662,  0.18162225,  1.20302   ],
        [-0.2699838 , -0.232636  , -0.16886197,  0.80234945],
        [-0.06377809,  0.29177746,  0.2267061 ,  0.83272314],
        [-0.13837768,  0.00836201,  0.13462907,  1.4371006 ],
        [ 0.26495618,  0.09185651, -0.02593795,  0.78029126],
        [-0.32194087, -0.22268036,  0.04040026,  0.7900227 ],
        [-0.27107474,  0.21034245, -0.10087233,  0.7808461 ],
        [ 0.2499975 ,  0.18103234, -0.20780838,  0.75521183],
        [-0.11619762, -0.03932963, -0.21887048,  1.2198398 ],
        [ 0.3659084 , -0.15836214,  0.04481811,  0.82320106]],
       dtype=float32)>,
 <tf.Tensor: shape=(10, 4), dtype=float32, numpy=
 array([[-0.09368034,  0.34990695, -0.2862136 ,  0.87195563],
        [-0.21898794,  0.2936428 , -0.11192943,  0.8356592 ],
        [ 0.12116119, -0.03710902, -0.06740739,  1.4182018 ],
        [-0.09688495,  0.07768627, -0.12889177,  1.3306088 ],
       

In [10]:
#Initialize Gaussians to random values with amplitudes over a narrow range
rnd=tf.random.uniform((500,4))     # specify the number of Gaussians to start with here
rnd+=(-.5,-.5,-.5,100.0)
gaus._data=rnd/(1.5,1.5,1.5,100.0)

# initial downsampling
ptclsfds=ptcls.downsample(16)    # downsample specifies the final size, not the amount of downsampling
ny=ptclsfds.shape[1]

In [11]:
# "learn" Gaussian locations test on downsampled particles
for i in range(64):
    with tf.GradientTape() as gt:
        gt.watch(gaus.tensor)
        projs=gaus.project_simple(orts,ny,txty=txty)
        projsf=projs.do_fft()
        frcs=tf_frc(projsf.tensor,ptclsfds.tensor,8,1.5)
    grad=gt.gradient(frcs,gaus._data)
    qual=tf.math.reduce_mean(frcs)
    shift=tf.math.reduce_std(grad[:,:3])
    sca=tf.math.reduce_std(grad[:,3])
    xyzs=1.0/(shift*1000)   # xyz scale
    gaus.add_tensor(grad*(xyzs,xyzs,xyzs,1.0/(sca*500)))
    print(f"{i}) {float(qual)}\t{float(shift)}\t{float(sca)}")

print(tf.reduce_min(gaus.tensor,0),tf.reduce_max(gaus.tensor,0),tf.reduce_mean(gaus.tensor,0))
vol=gaus.volume(map1["nx"])
vol.emdata[0].process_inplace("filter.lowpass.gauss",{"cutoff_abs":0.3})
vol.write_images("A_vol_opt_1.hdf")

0) 0.35528624057769775	4.914833068847656	0.3834434747695923
1) 0.39192450046539307	4.833830833435059	0.3878040909767151
2) 0.4279410243034363	4.733438968658447	0.3905066251754761
3) 0.463177889585495	4.619595527648926	0.3918691575527191
4) 0.4975927770137787	4.504326343536377	0.3922547399997711
5) 0.5312004685401917	4.383525371551514	0.3915213644504547
6) 0.563849687576294	4.2362236976623535	0.38969215750694275
7) 0.5952832102775574	4.055760860443115	0.3866572976112366
8) 0.6253888607025146	3.8703832626342773	0.38236889243125916
9) 0.6540690660476685	3.6701273918151855	0.37700334191322327
10) 0.6812769770622253	3.4610109329223633	0.37084513902664185
11) 0.7069774866104126	3.2677650451660156	0.36413127183914185
12) 0.7313504815101624	3.0825955867767334	0.35705700516700745
13) 0.7543268203735352	2.888688087463379	0.34980344772338867
14) 0.7758808732032776	2.7074086666107178	0.34238022565841675
15) 0.7961642742156982	2.5364136695861816	0.33493098616600037
16) 0.8152087330818176	2.36510419

In [12]:
print(len(gaus))
gaus.norm_filter(thr=.75)
print(len(gaus))
gaus.replicate(4,.02)
print(len(gaus))

ptclsfds=ptcls.downsample(32)    # downsample specifies the final size, not the amount of downsampling
ny=ptclsfds.shape[1]

500
217
868


In [13]:
# "learn" Gaussian locations test on downsampled particles
for i in range(64):
    with tf.GradientTape() as gt:
        gt.watch(gaus.tensor)
        projs=gaus.project_simple(orts,ny,txty=txty)
        projsf=projs.do_fft()
        frcs=tf_frc(projsf.tensor,ptclsfds.tensor,16,1.5)
    grad=gt.gradient(frcs,gaus._data)
    qual=tf.math.reduce_mean(frcs)
    shift=tf.math.reduce_std(grad[:,:3])
    sca=tf.math.reduce_std(grad[:,3])
    xyzs=1.0/(shift*1000)   # xyz scale
    gaus.add_tensor(grad*(xyzs,xyzs,xyzs,1.0/(sca*500)))
    print(f"{i}) {float(qual)}\t{float(shift)}\t{float(sca)}")

print(tf.reduce_min(gaus.tensor,0),tf.reduce_max(gaus.tensor,0),tf.reduce_mean(gaus.tensor,0))
vol=gaus.volume(map1["nx"])
vol.emdata[0].process_inplace("filter.lowpass.gauss",{"cutoff_abs":0.35})
vol.write_images("A_vol_opt_2.hdf")

0) 0.6994509696960449	4.0980682373046875	0.11501022428274155
1) 0.750622034072876	4.022082328796387	0.11405118554830551
2) 0.8002968430519104	3.859147310256958	0.11160175502300262
3) 0.8476165533065796	3.6389060020446777	0.10791453719139099
4) 0.891668975353241	3.342344284057617	0.10331334918737411
5) 0.9317454695701599	3.0028598308563232	0.09793845564126968
6) 0.9673332571983337	2.6322193145751953	0.09218120574951172
7) 0.9982870221138	2.2589635848999023	0.08611619472503662
8) 1.0245095491409302	1.8916376829147339	0.07973641902208328
9) 1.0464437007904053	1.5716102123260498	0.07323280721902847
10) 1.0646402835845947	1.2952909469604492	0.06677637994289398
11) 1.0795844793319702	1.0565744638442993	0.06060481444001198
12) 1.0917764902114868	0.8563822507858276	0.05493168905377388
13) 1.1017041206359863	0.6942606568336487	0.049914464354515076
14) 1.1097604036331177	0.5627877712249756	0.04528721794486046
15) 1.1162859201431274	0.4530074894428253	0.041202399879693985
16) 1.121507167816162	0.

In [14]:
print(len(gaus))
gaus.norm_filter(thr=.6)
print(len(gaus))
gaus.replicate(4,.01)
print(len(gaus))

ptclsfds=ptcls.downsample(64)    # downsample specifies the final size, not the amount of downsampling
ny=ptclsfds.shape[1]

868
788
3152


In [15]:
# "learn" Gaussian locations test on downsampled particles
for i in range(64):
    with tf.GradientTape() as gt:
        gt.watch(gaus.tensor)
        projs=gaus.project_simple(orts,ny,txty=txty)
        projsf=projs.do_fft()
        frcs=tf_frc(projsf.tensor,ptclsfds.tensor,-1)
    grad=gt.gradient(frcs,gaus._data)
    qual=tf.math.reduce_mean(frcs)
    shift=tf.math.reduce_std(grad[:,:3])
    sca=tf.math.reduce_std(grad[:,3])
    xyzs=1.0/(shift*1000)   # xyz scale
    gaus.add_tensor(grad*(xyzs,xyzs,xyzs,1.0/(sca*500)))
    print(f"{i}) {float(qual)}\t{float(shift)}\t{float(sca)}")

print(tf.reduce_min(gaus.tensor,0),tf.reduce_max(gaus.tensor,0),tf.reduce_mean(gaus.tensor,0))
vol=gaus.volume(map1["nx"])
vol.emdata[0].process_inplace("filter.lowpass.gauss",{"cutoff_abs":0.4})
vol.write_images("A_vol_opt_3.hdf")
#vol.emdata[0].get_attr_dict()


0) 0.3859594166278839	2.440650224685669	0.028969749808311462
1) 0.4906199872493744	2.1578736305236816	0.028484351933002472
2) 0.5764555931091309	1.818275809288025	0.027116145938634872
3) 0.6508519649505615	1.5383440256118774	0.025302255526185036
4) 0.7118127942085266	1.269783616065979	0.02362540178000927
5) 0.7569211721420288	1.0721246004104614	0.021973807364702225
6) 0.7892785668373108	0.9720284342765808	0.020552393049001694
7) 0.8116066455841064	1.0587717294692993	0.019471026957035065
8) 0.8238651156425476	1.037666916847229	0.018404686823487282
9) 0.8367001414299011	1.0283383131027222	0.017285548150539398
10) 0.843501091003418	1.047367811203003	0.016575338318943977
11) 0.8532236814498901	1.054111361503601	0.015878671780228615
12) 0.8567171096801758	1.0316859483718872	0.015229920856654644
13) 0.8640272617340088	1.0460065603256226	0.01471234206110239
14) 0.8644266724586487	1.0567573308944702	0.014246844686567783
15) 0.8706501722335815	1.0368391275405884	0.01400828268378973
16) 0.870177

In [7]:
projs.write_images("A_reproj_opt.hdf")

In [8]:
vol=gaus.volume(map1["nx"])
vol.emdata[0].process_inplace("filter.lowpass.gauss",{"cutoff_abs":0.2})
vol.write_images("A_vol_opt_1.hdf")
vol.emdata[0].get_attr_dict()

{'all_int': 0,
 'apix_x': 1.0,
 'apix_y': 1.0,
 'apix_z': 1.0,
 'changecount': 10,
 'datatype': 7,
 'is_complex': 0,
 'is_complex_ri': 0,
 'is_fftodd': 0,
 'is_fftpad': 0,
 'maximum': 0.04981573671102524,
 'mean': 4.7619989345548674e-05,
 'mean_nonzero': 4.762176104122773e-05,
 'minimum': -6.521835530293174e-06,
 'npad': 1,
 'nx': 128,
 'ny': 128,
 'nz': 128,
 'sigma': 0.0007934210007078946,
 'sigma_nonzero': 0.0007934356690384448,
 'square_sum': 1.324947476387024}

In [19]:
t=Transform()
tuple(t.get_trans_2d())

(0.0, 0.0)

In [17]:
io=tf.io.serialize_tensor(gaus.tensor)

In [32]:
gaus.tensor

<tf.Tensor: shape=(3152, 4), dtype=float32, numpy=
array([[-0.04800028, -0.06181109,  0.06013177,  0.7327289 ],
       [-0.1670731 ,  0.10643263,  0.17440087,  0.7102392 ],
       [ 0.14954612, -0.12320649, -0.08890869,  0.67823684],
       ...,
       [ 0.11189762,  0.07044013,  0.19778779,  0.8038612 ],
       [ 0.1142301 ,  0.07582243, -0.21208015,  0.75677645],
       [-0.0058985 ,  0.01942485,  0.04024125,  0.88480234]],
      dtype=float32)>

In [29]:
gaus.tensor.shape

TensorShape([3152, 4])

In [34]:
tf.io.write_file("test.tf",io)

In [39]:
io2=tf.io.read_file("test.tf")
ary=tf.io.parse_tensor(io2,tf.float32)
ary

<tf.Tensor: shape=(3152, 4), dtype=float32, numpy=
array([[-0.04800028, -0.06181109,  0.06013177,  0.7327289 ],
       [-0.1670731 ,  0.10643263,  0.17440087,  0.7102392 ],
       [ 0.14954612, -0.12320649, -0.08890869,  0.67823684],
       ...,
       [ 0.11189762,  0.07044013,  0.19778779,  0.8038612 ],
       [ 0.1142301 ,  0.07582243, -0.21208015,  0.75677645],
       [-0.0058985 ,  0.01942485,  0.04024125,  0.88480234]],
      dtype=float32)>